In [1]:
import os

print(os.getcwd())

E:\Code\bioML\final


In [2]:
from transformers import AutoImageProcessor

model_name = './image_aug_model_epoch_10_01-07-24-16-05-41'
processor = AutoImageProcessor.from_pretrained(model_name)

In [3]:
from transformers import pipeline

classifier = pipeline("image-classification", model=model_name, tokenizer=processor, device=0)

In [4]:
labels = [
    'Healthy',
    'Atelectasis',
    'Consolidation',
    'Infiltration',
    'Pneumothorax',
    'Edema',
    'Emphysema',
    'Fibrosis',
    'Effusion',
    'Pneumonia',
    'Pleural_Thickening',
    'Cardiomegaly',
    'Nodule',
    'Mass',
    'Hernia'
]

In [5]:
import datasets

def top_k_accuracy(data, label, pipe):
    top_1 = 0
    top_3 = 0
    top_5 = 0
    for o in pipe(data):
        for _ in range(5):
            if label == o[_]["label"]:
                if _ < 1:
                    top_1 += 1
                if _ < 3:
                    top_3 += 1
                if _ < 5:
                    top_5 += 1
                break
    return top_1 / len(data), top_3 / len(data), top_5 / len(data)

In [6]:
import glob

res = dict()

for label in labels:
    ds = datasets.load_dataset("imagefolder", data_dir="./sample_dataset/dataset/" + label)['train']['image']
    res[label] = top_k_accuracy(ds, label, classifier)

Resolving data files:   0%|          | 0/3044 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/508 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/226 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/967 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/271 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/176 [00:00<?, ?it/s]

E:\Code\bioML\venv\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Resolving data files:   0%|          | 0/141 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/313 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/284 [00:00<?, ?it/s]

In [7]:
res

{'Healthy': (0.9336399474375822, 0.9944152431011827, 0.9993429697766097),
 'Atelectasis': (0.1377952755905512, 0.7244094488188977, 0.937007874015748),
 'Consolidation': (0.01327433628318584,
  0.12831858407079647,
  0.5309734513274337),
 'Infiltration': (0.22750775594622544, 0.8148914167528438, 0.9896587383660806),
 'Pneumothorax': (0.0996309963099631, 0.42066420664206644, 0.8118081180811808),
 'Edema': (0.0, 0.05084745762711865, 0.2711864406779661),
 'Emphysema': (0.0, 0.0, 0.03937007874015748),
 'Fibrosis': (0.0, 0.0, 0.0),
 'Effusion': (0.33229813664596275, 0.8493788819875776, 0.9782608695652174),
 'Pneumonia': (0.0, 0.0, 0.0),
 'Pleural_Thickening': (0.0, 0.005681818181818182, 0.028409090909090908),
 'Cardiomegaly': (0.0, 0.0, 0.02127659574468085),
 'Nodule': (0.0, 0.1853035143769968, 0.4952076677316294),
 'Mass': (0.01056338028169014, 0.18309859154929578, 0.4894366197183099),
 'Hernia': (0.0, 0.0, 0.0)}

In [8]:
import csv

with open(model_name + '.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['label', 'top_1', 'top_3', 'top_5'])
    for label in labels:
        writer.writerow([label, res[label][0], res[label][1], res[label][2]])